In [4]:
import datafetching, datetime, json
import pandas as pd
from google.cloud import storage

/Library/Python/3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [5]:
import os
os.environ.setdefault("GCLOUD_PROJECT", "disco-sky-405321")

'disco-sky-405321'

In [6]:
# GCS
client = storage.Client()
bucket = client.get_bucket('gw2pricedata-dump')
item_description_location = "items/item_description/"

In [7]:
blob = bucket.blob(item_description_location + "item_description2023-11-18 00_45_27")

In [63]:
data = json.loads(blob.download_as_string(client=None))

dict

In [63]:
item_desc_df = pd.DataFrame(data)
item_desc_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65925 entries, 0 to 65924
Data columns (total 50 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   name                                    65925 non-null  object 
 1   type                                    65925 non-null  object 
 2   level                                   65925 non-null  int64  
 3   rarity                                  65925 non-null  object 
 4   vendor_value                            65925 non-null  int64  
 5   default_skin                            35753 non-null  float64
 6   game_types                              65925 non-null  object 
 7   flags                                   65925 non-null  object 
 8   restrictions                            65925 non-null  object 
 9   id                                      65925 non-null  int64  
 10  chat_link                               65925 non-null  ob

<class 'pandas.core.frame.DataFrame'>
Index: 65925 entries, 0 to 0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    65925 non-null  object
dtypes: object(1)
memory usage: 1.0+ MB
